
_______________________________________________________________________________


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import pandas as pd
import numpy as np
from pandas import read_csv

In [ ]:
os.chdir('/content/gdrive/MyDrive/Colab Notebooks/УЛИТКА/Классификация/Сегментация_клиентов')

In [ ]:
filename='Train_cleared.csv'
df=read_csv(filename)
df=df.filter(regex='^(?!.*Unnamed).*$')
df = df.dropna()

In [ ]:
df.head(2)

,Пол,Женатость/замужность,Возраст,Высшее_образование,Профессия,Опыт_работы,Уровень_расходов,Размер_семьи,Анонимная_категория,Целевая_категория
0,1,0.0,22,0.0,2.0,1.0,3,4.0,4.0,4
2,0,1.0,67,1.0,4.0,1.0,3,1.0,6.0,2


_______________________________________________________________________

In [ ]:
# функции для разбивки данных на обучающую и тестовую:
from sklearn.model_selection import train_test_split # разбивка выборки начасти
from sklearn.linear_model import LogisticRegression # логистическая регрессия
from sklearn.model_selection import KFold # разбивка на фолды
from sklearn.model_selection import cross_val_score # для кросс валидации
from sklearn.model_selection import LeaveOneOut # оставляет одно значение для обученияб а остальные для тестов
from sklearn.model_selection import ShuffleSplit # случайная разбивка выборки на k частей
# функции для оценки качества данных:
from sklearn.metrics import confusion_matrix # сопоставляются расчетные значения
from sklearn.metrics import classification_report # тут разные показатели оценки
from sklearn.linear_model import LinearRegression # для оценки качества регрессии

In [ ]:
array = df.values
array

In [ ]:
X = array[:,0:9]
Y = array[:,9]

__________________________________________

In [ ]:
# стандартизация (при необходимости) (здесь не дала эффект)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler=StandardScaler().fit(X) # создали объект и обучили его

In [ ]:
rescaledX=scaler.transform(X) # делаем среднее =0, а стандартное отклонение = 1

___________________________________________

In [ ]:
test_size=0.33
seed = 7 # задается любое число, но всегда одинаковое, чтобы разбивка была одна и та же

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=test_size,random_state=seed)

solver (решатель){‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, default=’lbfgs’

Алгоритм для использования в задаче оптимизации. По умолчанию используется ‘lbfgs’. Чтобы выбрать решатель, вы можете рассмотреть следующие аспекты:

Для небольших наборов данных "liblinear" - хороший выбор, тогда как ‘sag’ и ‘saga’ быстрее для больших;

Для многоклассовых задач только ‘newton-cg’, ‘sag’, ‘saga’ и ‘lbfgs’ обрабатывают многочленные потери;

‘liblinear" ограничен схемами "один против другого".

**1-й вариант:**

In [ ]:
model=LogisticRegression(solver='newton-cg',class_weight= 'balanced')
model.fit(X_train,Y_train)
result=model.score(X_test,Y_test)
result

0.4740909090909091

**2-й вариант:**

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold=KFold(n_splits=10,random_state=7,shuffle=True)
model=LogisticRegression(solver='liblinear')
results=cross_val_score(model, X, Y, cv=kfold)
results.mean(), results.std()

(0.47636429532981256, 0.02705725760997702)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold=KFold(n_splits=3,random_state=7,shuffle=True) # делим выборку на части
model=LogisticRegression(solver='newton-cg',class_weight= 'balanced')
results=cross_val_score(model, X, Y, cv=kfold) # это обучение выборки разбитой на фолды
results.mean(), results.std()

(0.4798188283483909, 0.009518886933667982)

**3-й вариант (циклы):**

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [ ]:
lst_1 = list(range(1,8))
lst_2 = list(range(2,8))
k = []

In [ ]:
# логистическая_регрессия
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(solver='newton-cg',class_weight= 'balanced')    # максимальное значение: n_splits: 6 KBest: 7 0.4814692280038815
for i in lst_2:
   kfold=KFold(n_splits=i,random_state=7,shuffle=True)
   for j in lst_1:
      test=SelectKBest(score_func=f_classif, k=j)
      fit=test.fit(X,Y)
      features=fit.transform(X)
      results=cross_val_score(model, features, Y, cv=kfold)
      res = results.mean()
      k.append(res)
      print(f'n_splits: {i} KBest: {j} {res}')
   print(f'i {i} max: {max(k)}')

**ShuffleSplit:**

In [ ]:
# логистическая_регрессия
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ShuffleSplit
model=LogisticRegression(solver='newton-cg',class_weight= 'balanced')    # максимальное значение: n_splits: 5 KBest: 7 0.4807272727272728
for i in lst_2:
   kfold=ShuffleSplit(n_splits=i,random_state=7,test_size=0.33)
   for j in lst_1:
      test=SelectKBest(score_func=f_classif, k=j)
      fit=test.fit(X,Y)
      features=fit.transform(X)
      results=cross_val_score(model, features, Y, cv=kfold)
      res = results.mean()
      k.append(res)
      print(f'n_splits: {i} KBest: {j} {res}')
   print(f'i {i} max: {max(k)}')

____________________________________________________________

**Task_6_учебы:**

**Матрица ошибок**

In [ ]:
model = LogisticRegression(solver='newton-cg',class_weight= 'balanced').fit(X,Y)

In [ ]:
predicted = model.predict(X)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y,predicted)

array([[ 633,  242,  321,  420],
       [ 449,  299,  625,  199],
       [ 196,  240, 1060,  224],
       [ 345,  102,  103, 1207]])

In [ ]:
print(classification_report(Y,predicted))

              precision    recall  f1-score   support

         1.0       0.39      0.39      0.39      1616
         2.0       0.34      0.19      0.24      1572
         3.0       0.50      0.62      0.55      1720
         4.0       0.59      0.69      0.63      1757

    accuracy                           0.48      6665
   macro avg       0.46      0.47      0.46      6665
weighted avg       0.46      0.48      0.46      6665



________________________________________________________